## Imports

In [2]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Embedding, LSTM, Input, Concatenate

C:\Users\huday\AppData\Local\Temp\ipykernel_9812\249373202.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Data

In [38]:
df = pd.read_csv('./data.csv')
data = df[['num1', 'num2']]
label = df['label'].values

data = data.values.reshape(400, 2)

X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=42)
"""
# Split the data into training and testing sets
X_train = data[:, :320, :]  # Take first 320 samples for training
X_test = data[:, 320:, :]  # Take last 80 samples for testing
y_train = label[:320]  # Corresponding labels for training samples
y_test = label[320:]  # Corresponding labels for testing samples
"""
X_test.shape

(80, 2)

## History Data

In [5]:
# Load embeddings (assuming they are already loaded)
df_smell = pd.read_csv("./embeddings/Temp_smellKey-data_encoded.csv")
embeddings = torch.load('./embeddings/bert_old.pt')
embeddings2 = torch.load('./embeddings/bert_fixed.pt')
labels = df_smell['smellKey_encoded'].values
print(labels)
print(embeddings.shape, embeddings2.shape, labels.shape)
# Combine embeddings into pairs
X = torch.stack((embeddings, embeddings2), dim=1)  # Shape: (N, 2, 768)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)


[ 0  6 19 19 12 12 21  5  5  5  6  0 17 13  0  0  0  0  0 16 11  7 16 16
  0  0  7 20  0  8 25 25 26 25 25 25 25 25 25  3 26 25 26 25 26 25 25 25
 25 25 25 25 25 25 25 25 25 25 25 25  4  2 10 22  9 23  9 18 23 23 22 23
 23 23 23 23 11  9  9 23  3 23 22 23 25 23  9 23 24  9  9 23  1 24  1 24
 24 22 22 23  9  9 23 22 13 23 23 10  9 23 23 23 23  9 24 24 10 23 24 23
 23 11 23 15 23 23 23 23 23 23 23 24 23 24 14 23 24  9 22 10]
torch.Size([140, 768]) torch.Size([140, 768]) (140,)


In [7]:
model = Sequential()
model.add(LSTM(32, input_shape=(140, 2)))
model.add(Dense(1))

In [8]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(data, label.values)
print("Test Accuracy:", accuracy)

ValueError: `validation_split` is only supported for Tensors or NumPy arrays, found following types in the input: [<class 'torch.Tensor'>]

In [61]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Read the CSV file into a DataFrame
df = pd.read_csv('./data.csv')

# Extract features and labels
data = df[['num1', 'num2']]
label = df['label'].values
                    #data size,time step ,number of feature each step
data = data.values.reshape(140, 2, 1)
#400,2,1

X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=42)

model = Sequential()
model.add(LSTM(32, input_shape=(2, 1)))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=25, batch_size=20, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)


Epoch 1/25
13/13 [==============================] - 2s 40ms/step - loss: 0.7000 - accuracy: 0.6641 - val_loss: 0.6777 - val_accuracy: 0.6719
Epoch 2/25
13/13 [==============================] - 0s 5ms/step - loss: 0.6453 - accuracy: 0.6914 - val_loss: 0.6435 - val_accuracy: 0.6562
Epoch 3/25
13/13 [==============================] - 0s 5ms/step - loss: 0.6219 - accuracy: 0.6797 - val_loss: 0.6135 - val_accuracy: 0.6562
Epoch 4/25
13/13 [==============================] - 0s 6ms/step - loss: 0.5943 - accuracy: 0.6797 - val_loss: 0.5944 - val_accuracy: 0.6562
Epoch 5/25
13/13 [==============================] - 0s 5ms/step - loss: 0.5755 - accuracy: 0.6875 - val_loss: 0.5734 - val_accuracy: 0.6719
Epoch 6/25
13/13 [==============================] - 0s 5ms/step - loss: 0.5569 - accuracy: 0.6992 - val_loss: 0.5556 - val_accuracy: 0.6875
Epoch 7/25
13/13 [==============================] - 0s 6ms/step - loss: 0.5479 - accuracy: 0.6953 - val_loss: 0.5362 - val_accuracy: 0.6875
Epoch 8/25
13/13 [=

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Softmax

# Read the CSV file into a DataFrame
df = pd.read_csv('./data.csv')

# Extract features and labels
data = df[['num1', 'num2']]
label = df['label'].values
                    #data size,time step ,number of feature each step
data = data.values.reshape(400, 2, 1)
#400,2,1

X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=42)

model = Sequential()
model.add(LSTM(32, input_shape=(2, 1)))
model.add(Dense(1))
model.add(Dense(units=1, activation='ReLU'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=20, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)

Epoch 1/100
13/13 [==============================] - 3s 48ms/step - loss: 2.5268 - accuracy: 0.3281 - val_loss: 1.0336 - val_accuracy: 0.3438
Epoch 2/100
13/13 [==============================] - 0s 8ms/step - loss: 0.8657 - accuracy: 0.3320 - val_loss: 0.7406 - val_accuracy: 0.5156
Epoch 3/100
13/13 [==============================] - 0s 8ms/step - loss: 0.7111 - accuracy: 0.6094 - val_loss: 0.7022 - val_accuracy: 0.5938
Epoch 4/100
13/13 [==============================] - 0s 8ms/step - loss: 0.6900 - accuracy: 0.6211 - val_loss: 0.7041 - val_accuracy: 0.6250
Epoch 5/100
13/13 [==============================] - 0s 9ms/step - loss: 0.6837 - accuracy: 0.6289 - val_loss: 0.6914 - val_accuracy: 0.6250
Epoch 6/100
13/13 [==============================] - 0s 8ms/step - loss: 0.6754 - accuracy: 0.6289 - val_loss: 0.6831 - val_accuracy: 0.6250
Epoch 7/100
13/13 [==============================] - 0s 8ms/step - loss: 0.6685 - accuracy: 0.6289 - val_loss: 0.6741 - val_accuracy: 0.6250
Epoch 8/100


In [11]:
df = pd.read_csv('./embeddings/Temp_smellKey-data_encoded.csv')
message_encoded = df['smellKey_encoded']
#save pt file   
torch.save(message_encoded, 'message_encoded.pt')
#load pt file
message_encoded = torch.load('message_encoded.pt')
message_encoded

0       0
1       6
2      19
3      19
4      12
       ..
135    23
136    24
137     9
138    22
139    10
Name: smellKey_encoded, Length: 140, dtype: int64

In [13]:
# Combine the embeddings and encoded messages into a DataFrame
import torch
import numpy as np
embeddings = torch.load('embeddings/bert_old.pt')
embeddings2 = torch.load('embeddings/bert_old.pt')
embeddings3 = torch.load('embeddings/bert_fixed.pt')
data = pd.DataFrame({
    'fixdd_embeddings.pt': embeddings3.tolist(),
    'old_embeddings.pt': embeddings2.tolist(),
    'smellKey_encoded': message_encoded
})


In [14]:
x1 =np.vstack(data['fixdd_embeddings.pt'])
x2= np.vstack(data['old_embeddings.pt'])
X = np.concatenate((x1, x2), axis=1)
X.shape

(140, 1536)

In [15]:

# Split the data into features (embeddings) and labels (messages) tensor

y = (data['smellKey_encoded'])
X= X.reshape(140, 1, 1536)

tensor = tf.convert_to_tensor(X, dtype=tf.float32)


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [16]:
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential()
model.add(LSTM(512, input_shape=(1, 1536)))  # Increase LSTM units
model.add(Dropout(0.5))  # Add dropout layer to prevent overfitting
model.add(Dense(units=1, activation='ReLU'))  # Change activation function to ReLU
model.add(Dropout(0.2))  # Add dropout layer to prevent overfitting
model.add(Dense(units=1, activation='softmax'))  # Change activation function to ReLU

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=20, validation_split=0.2)

# Evaluate the model

#X_train convert numpy to tensor

loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)

Epoch 1/50
5/5 [==============================] - 3s 176ms/step - loss: 0.7056 - accuracy: 0.0112 - val_loss: 0.6163 - val_accuracy: 0.0435
Epoch 2/50
5/5 [==============================] - 0s 51ms/step - loss: 0.5785 - accuracy: 0.0112 - val_loss: 0.5396 - val_accuracy: 0.0435
Epoch 3/50
5/5 [==============================] - 0s 50ms/step - loss: 0.4927 - accuracy: 0.0112 - val_loss: 0.4636 - val_accuracy: 0.0435
Epoch 4/50
5/5 [==============================] - 0s 51ms/step - loss: 0.4077 - accuracy: 0.0112 - val_loss: 0.3880 - val_accuracy: 0.0435
Epoch 5/50
5/5 [==============================] - 0s 50ms/step - loss: 0.3208 - accuracy: 0.0112 - val_loss: 0.3129 - val_accuracy: 0.0435
Epoch 6/50
5/5 [==============================] - 0s 52ms/step - loss: 0.2380 - accuracy: 0.0112 - val_loss: 0.2367 - val_accuracy: 0.0435
Epoch 7/50
5/5 [==============================] - 0s 52ms/step - loss: 0.1525 - accuracy: 0.0112 - val_loss: 0.1598 - val_accuracy: 0.0435
Epoch 8/50
5/5 [==========

TypeError: 'builtin_function_or_method' object cannot be interpreted as an integer